<a href="https://colab.research.google.com/github/kongminhyuk/swboot_camp/blob/main/base_model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Install
warning은 무시해도 됩니다
임의로 버전을 고정한거라 뜨는 메세지 같아요

In [ ]:
!pip install --upgrade setuptools #Python 프로젝트를 패키징하고 배포하며 설치하는 프로세스를 용이하게 하는 패키지입니다.
!pip install transformers==4.16.2 #자연어 처리를 위한 라이브러리로, 사전 훈련된 모델, 파인튜닝 도구 및 다양한 유틸리티를 제공합니다.
!pip install sentencepiece #신경망 기반 텍스트 생성 작업에 특화된 토큰화를 위한 라이브러리입니다.
!pip install datasets
!pip install rouge_score #ROUGE 스코어를 계산하기 위한 라이브러리로, 자연어 처리 작업에서 자주 사용됩니다.
!pip install pytorch_lightning==1.5.10 #고성능 신경망 훈련을 위한 PyTorch 래퍼입니다.

  Using cached setuptools-68.2.2-py3-none-any.whl (807 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.5.0
    Uninstalling setuptools-59.5.0:
      Successfully uninstalled setuptools-59.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
pytorch-lightning 1.5.10 requires setuptools==59.5.0, but you have setuptools 68.2.2 which is incompatible.


  Using cached setuptools-59.5.0-py3-none-any.whl (952 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 68.2.2
    Uninstalling setuptools-68.2.2:
      Successfully uninstalled setuptools-68.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
arviz 0.15.1 requires setuptools>=60.0.0, but you have setuptools 59.5.0 which is incompatible.
cvxpy 1.3.2 requires setuptools>65.5.1, but you have setuptools 59.5.0 which is incompatible.


In [ ]:
!git clone https://github.com/lbox-kr/lbox_open.git --branch v0.1
%cd lbox_open

Cloning into 'lbox_open'...
remote: Enumerating objects: 266, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 266 (delta 128), reused 210 (delta 94), pack-reused 0
Receiving objects: 100% (266/266), 86.64 KiB | 677.00 KiB/s, done.
Resolving deltas: 100% (128/128), done.
Note: switching to '5dd718fe2424d84bc73a78890b357b429c2f9cb0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

/content/lbox_open


In [ ]:
import os
from argparse import Namespace

import torch
import transformers
import pytorch_lightning as pl

from lcube.data_module.data_lbox_open import LBoxOpenDataModule
from lcube.model.model_baseline import SeqToSeqBaseline

device = 'cuda' if torch.cuda.is_available() else "cpu"

#2.Parameter 생성

In [ ]:
args = Namespace()
# dataset
args.dataset_card = "lbox/lbox_open" #데이터셋 카드를 지정하는데, 현재 "lbox/lbox_open"으로 설정되어 있는 것으로 보입니다.
args.task = "casename_classification"  # 작업 유형을 지정하는데, 케이스 이름 분류, 법률 분류 또는 요약의 옵션
# args.task = "statute_classification"
# args.task = "summarization"


if args.task in ["casename_classification", "statute_classification"]:
    args.input_key = "facts"

    # model
    args.model_card = "google/mt5-small" #모델 카드를 "google/mt5-small"으로 설정
    args.max_input_len = 512
    args.max_target_len = 64

    # train
    args.max_epochs = 10
    args.learning_rate = 2e-4
    args.batch_size = 8
    args.batch_size_eval = 2 * args.batch_size
    args.accumulate_grad_batches = 1
    args.validation_metric = "exact_match"

elif args.task == "summarization":
    args.input_key = "precedent"

    # model
    args.model_card = "google/mt5-small"
    args.max_input_len = 768
    args.max_target_len = 512

    # train
    args.max_epochs = 10
    args.learning_rate = 2e-4
    args.batch_size = 1
    args.batch_size_eval = 2 * args.batch_size
    args.accumulate_grad_batches = 8
    args.validation_metric = "rougeL"

else:
    raise ValueError


args.tokenizer = transformers.MT5TokenizerFast.from_pretrained(args.model_card)
pl.seed_everything(seed=1, workers=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:pytorch_lightning.utilities.seed:Global seed set to 1


1

# 3. Datasets 불러오기

In [ ]:
data_module = LBoxOpenDataModule(
    args.dataset_card,
    args.task,
    args.tokenizer,
    args.max_input_len,
    args.max_target_len,
    args.batch_size,
    args.batch_size_eval,
)

# 4. 모델 준비
MT5ForConditionalGeneration

In [ ]:
backbone = transformers.MT5ForConditionalGeneration.from_pretrained(args.model_card)
model = SeqToSeqBaseline(
    args.task,
    backbone,
    args.tokenizer,
    args.learning_rate,
    args.max_target_len,
    args.validation_metric

)

# 5. 학습 준비
epoch 5~10 정도 해야할듯 합니다




## 성능 검증 그래프화

In [ ]:
import matplotlib.pyplot as plt
from pytorch_lightning import LightningModule, Trainer, callbacks

class PlotLossesCallback(callbacks.Callback):
    def __init__(self):
        self.train_losses = []
        self.val_losses = []

    def on_epoch_end(self, trainer, pl_module):
        # 에폭 종료 시 호출되며 훈련 및 검증 손실을 기록합니다.
        self.train_losses.append(trainer.callback_metrics.get("train_loss"))
        self.val_losses.append(trainer.callback_metrics.get("val_loss"))

        # 손실 그래프를 업데이트합니다.
        self.plot_losses()

    def plot_losses(self):
        plt.figure(figsize=(10, 5))
        plt.plot(self.train_losses, label='Training Loss', color='blue')
        plt.plot(self.val_losses, label='Validation Loss', color='red')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title('Training and Validation Loss')
        plt.legend()
        plt.show()

class YourLightningModule(LightningModule):
    def validation_step(self, batch, batch_idx):
        # 여기에 검증 단계 논리를 작성하세요
        outputs = self.forward(batch)

        # 예측과 정답 간의 손실 계산
        loss = your_loss_function(outputs, batch['target'])  # 적절한 손실 함수를 사용해야 합니다.

        # 검증 손실을 로그에 기록
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)

        return outputs

# ModelCheckpoint 콜백 및 그래프화 콜백을 설정합니다.
model_checkpoint = callbacks.ModelCheckpoint(monitor='val_loss', dirpath='./saved/0/{args.task}', save_top_k=1, mode='max')
plot_losses_callback = PlotLossesCallback()

In [ ]:
callbacks = pl.callbacks.ModelCheckpoint(
    monitor=args.validation_metric,
    dirpath=f"./saved/0/{args.task}",
    save_top_k=1,
    mode="max",
)
trainer = pl.Trainer(
    max_epochs = args.max_epochs,
    gpus=torch.cuda.device_count(),
    accumulate_grad_batches=args.accumulate_grad_batches,
    fast_dev_run=not True,
    callbacks=callbacks,
)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs


In [ ]:
#trainer = pl.Trainer(
 #   max_epochs=1,
 #   gpus=torch.cuda.device_count(),
 #   accumulate_grad_batches=args.accumulate_grad_batches,
 #   fast_dev_run=not True,
 #   callbacks=[model_checkpoint, plot_losses_callback]
)


INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs


In [ ]:
# PyTorch Lightning을 사용하여 모델을 훈련하고 지정된
# 검증 메트릭에 대한 최고 성능을 갖는 모델을 저장하기 위해
# ModelCheckpoint 콜백을 설정합니다.
#callbacks = pl.callbacks.ModelCheckpoint(
    #monitor=args.validation_metric,  # 검증 메트릭을 모니터링하여 최고 성능 모델을 찾습니다.
 #   monitor='val_loss',
  #  dirpath=f"./saved/0/{args.task}",  # 최고 성능 모델이 저장될 디렉토리입니다.
   # save_top_k=1,  # 최고의 모델 하나만 저장합니다.
   # mode="max",  # 모니터링 메트릭의 최대 값을 가진 모델을 저장합니다.
#)

# PyTorch Lightning Trainer 인스턴스를 생성합니다.
#trainer = pl.Trainer(
 #   max_epochs= 2,  # 최대 에폭 수는 1입니다. 더 많은 에폭으로 훈련하려면 10으로 주석 처리된 줄을 사용할 수 있습니다.
    #max_epochs = 10,
  #  gpus=torch.cuda.device_count(),  # 사용할 GPU 수를 설정합니다.
   # accumulate_grad_batches=args.accumulate_grad_batches,  # 그래디언트 누적 배치 수를 설정합니다.
    #fast_dev_run=not True,  # True로 설정하면 훈련 및 검증을 위해 하나의 배치만 실행됩니다.
    #callbacks=callbacks,  # 사용할 콜백을 설정합니다.

#)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs


# 6. 학습
평균 1회당 9분 정도 소요

In [ ]:
trainer.fit(model, data_module)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


OutOfMemoryError: ignored

# 7. Test
테스트는 input이 고정이여서 정확도 체크 용도로 쓰면 될듯합니다

In [ ]:
input_text = data_module.dataset["test"][40][args.input_key]

model_inputs = args.tokenizer(
            input_text,
            max_length=args.max_input_len,
            padding=True,
            truncation=True,
            return_tensors='pt',
        )
model_inputs = {k: v.to(device) for k,v in model_inputs.items()}

In [ ]:
model.model = model.model.to(device)
pr_seqs = model.model.generate(model_inputs["input_ids"], max_length=args.max_target_len)
prs = args.tokenizer.batch_decode(pr_seqs, skip_special_tokens=True)
print(f"Input\n {input_text}\n\n")
print(f"Prediction\n {prs}")

Input
 검색하실 판례를 입력해 주세요:


Prediction
 ['<extra_id_0>)무추서']


#8. input을 임의로 넣을 수 있는 코드 추가

In [ ]:
from transformers import MT5TokenizerFast, MT5ForConditionalGeneration

# 토크나이저 및 모델 초기화
tokenizer = MT5TokenizerFast.from_pretrained(args.model_card)
model = MT5ForConditionalGeneration.from_pretrained(args.model_card)

# 저장된 모델 불러오기
checkpoint_path = "./saved/0/{args.task}/epoch=2.ckpt"  # 여기에 실제로 저장된 모델의 경로를 지정하세요.
model.load_state_dict(torch.load(checkpoint_path)["state_dict"])

# 검색 기능 함수 정의
def search(query):
    inputs = tokenizer.encode(query, return_tensors="pt", max_length=args.max_input_len, truncation=True)
    outputs = model.generate(inputs, max_length=args.max_target_len, num_beams=4, length_penalty=2.0, early_stopping=True)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

# 검색어 입력 및 검색 수행
search_query = "검색하실 판례를 작성해주세요:"
search_result = search(search_query)
print("위법 법률:", search_result)

Input
 검색할 판례를 작성해주세요:


Prediction
 ['손해등이용nissen신청']
